In [162]:
import snowflake.connector
import csv
import pandas as pd
import random
import numpy as np

In [2]:
ctx = snowflake.connector.connect(
    user='ssamichya',
    password='Lisnepal@2021',
    account='em35717.ap-south-1.aws',
    )

cursor = ctx.cursor()

In [ ]:
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.STG")

cursor.execute("create or replace table COUNTRYSTG(ID int NOT NULL, Country varchar(200))")
cursor.execute("create or replace table REGIONSTG(ID int NOT NULL, CountryID int, Region varchar(200))")
cursor.execute("create or replace table STORESTG(ID int NOT NULL, RegionID int, Store varchar(200))")

#Get Country, Region and Store from BHATBHATENI_KEC_SURAJ
cursor.execute("use BHATBHATENI_KEC_SURAJ.TRANSACTIONS")

header_Country = ["Id", "Country"]
header_Region = ["Id", "CountryID", "Region"]
header_Store = ["ID", "RegionID", "Store"]

cursor.execute("select * from COUNTRY")
result = cursor.fetchall()

f = csv.writer(open('E:/Country.csv', 'w', newline=''))
f.writerow(header_Country)
for y in result:
    f.writerow(y)

cursor.execute("select * from REGION")
result = cursor.fetchall()

writer = csv.writer(open('E:/Region.csv', 'w',newline=''))
writer.writerow(header_Region)
for row in result:
    writer.writerow(row)

cursor.execute("select * from STORE")
result = cursor.fetchall()

f = csv.writer(open('E:/Store.csv', 'w', newline=''))
f.writerow(header_Store)
for row in result:
    f.writerow(row)


#Load Data from CSV to Staging Tables

#CountrySTG
data_country = pd.read_csv("E:/Country.csv")

cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.STG;")
for i, row in data_country.iterrows():
    query = "insert into CountrySTG values(%s,%s)"
    cursor.execute(query,tuple(row))

In [9]:
#RegionSTG
data_region = pd.read_csv("E:/Region.csv")
for i, row in data_region.iterrows():
    query = "insert into RegionSTG values(%s,%s,%s)"
    cursor.execute(query,tuple(row))

In [10]:
#StoreSTG
data_store = pd.read_csv("E:/Store.csv")
for i, row in data_store.iterrows():
    query = "insert into StoreSTG values(%s,%s,%s)"
    cursor.execute(query,tuple(row))

In [114]:
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.STG;")
hierarchy_query = "with LocationCTE AS(select CountrySTG.ID as CountryID,Country, Region,StoreSTG.ID as StoreID ,Store from CountrySTG inner join RegionSTG  on CountrySTG.ID = RegionSTG.CountryId inner join StoreSTG  on RegionSTG.ID = StoreSTG.REGIONID)select * from LocationCTE;"
cursor.execute(hierarchy_query)
result = cursor.fetchall()
writer = csv.writer(open('E:/Location_Hierarchy.csv','w'))
writer.writerow(["ID","Country","Region","StoreID","Store"])
for row in result:
    writer.writerow(row)

In [129]:
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.TMP;")

In [116]:
cursor.execute("create or replace table LocationTMP(ID int NOT NULL, Country varchar(50), Region varchar(50), StoreID int ,Store varchar(50))")

In [117]:
data_location = pd.read_csv("E:/Location_Hierarchy.csv")
for i,row in data_location.iterrows():
    cursor.execute("insert into LocationTMP values(%s,%s,%s,%s,%s)",tuple(row))

In [118]:
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.TGT;")


In [126]:
cursor.execute("create or replace table LocationTGT(ID int, Country varchar(50), Region varchar(50), StoreID int,Store varchar(50), start_date date, end_date date, Active char(1),RCD_INS_TS TIMESTAMP_LTZ(9), RCD_UPD_TS TIMESTAMP_LTZ(9))")

In [120]:
data_to_stg = pd.read_csv("E:/Location_Hierarchy.csv")

In [121]:
import datetime
ts = datetime.datetime.now()

In [103]:
def get_start_date():
    start = "2020-01-01"
    end = "2021-06-01"
    array_start = start.split("-")
    array_end = end.split("-")
    start_date = str(random.randint(int(array_start[0]),int(array_end[0]))) + "-" + str(random.randint(int(array_start[1]),int(array_end[1]))).zfill(2) + "-" + str(random.randint(int(array_start[2]),int(array_end[2]))).zfill(2)
    return  start_date

In [196]:
def get_end_date(id):
    if check_if_active(id) == 'Y':
          start = "2022-05-01"
          end = "2030-12-30"
          array_start = start.split("-")
          array_end = end.split("-")
          end_date = str(random.randint(int(array_start[0]),int(array_end[0]))) + "-" + str(random.randint(int(array_start[1]),int(array_end[1]))).zfill(2) + "-" + str(random.randint(int(array_start[2]),int(array_end[2]))).zfill(2)
          return  end_date
    else:
        return datetime.date.today()

In [193]:
def check_if_active(id):
    cursor.execute("select StoreID from BHATBHATENI_KEC_DWH_SURAJ.TMP.LocationTMP Where exists(select ID from BHATBHATENI_KEC_DWH_SURAJ.STG.StoreSTG);")
    result = cursor.fetchall()
    result_list = []
    activ = 'Y'
    for i in result:
        b = list(i)
        result_list.append(i)
    result_list = np.array(result_list)
    result = result_list.flatten()
    for i in result:
        if i == id:
            activ =  'Y'
            break
        else:
            activ = 'N'  
    return activ

In [197]:
for index, row in data_to_stg.iterrows():
    cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.TGT;")
    query = "insert into LocationTGT values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    storeid = row.StoreID
    cursor.execute(query,(index+1, row.Country, row.Region, storeid,row.Store, get_start_date(), get_end_date(storeid), check_if_active(storeid), ts, ts))